In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install -q tensorflow_gpu==2.0.0-beta1 tensorflow-datasets matplotlib tqdm tensorflow_hub scipy

import os


from data_generator import DataGenerator

import matplotlib.pyplot as plt

from numpy import array
import numpy as np


import datetime as dt
import datetime
from datetime import datetime

from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, LeakyReLU, Dropout, LSTM, GRU, Input
from tensorflow.keras import Model, Sequential
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator, pad_sequences


from tensorflow.keras import datasets, layers, models

import tensorflow_datasets as tfds
import tensorflow_hub as hub
import tensorflow as tf

from tensorboard.plugins.hparams import api as hp

%load_ext tensorboard
print(tf.__version__)

  ERROR: Could not find a version that satisfies the requirement tensorflow_gpu==2.0.0-beta1 (from versions: none)
ERROR: No matching distribution found for tensorflow_gpu==2.0.0-beta1
2.0.0-beta1


In [2]:
dt = DataGenerator(random=False, first_index=10)
dt.rewind()
dt.steps

155506

In [3]:
# integer encode input data
def onehot_encoded (integer_encoded, char_to_int = 2):
    # one hot encode
    onehot_encoded = list()
    letter = [0 for _ in range(char_to_int)]
    letter[integer_encoded] = 1
    onehot_encoded.append(letter)
    
    return onehot_encoded[0]


In [4]:
last_price = 0
last_time = 0
should_buy = 0
should_sell = 0

def get_date(state):
    timestamp = int(state["timestamp"])
    return datetime.fromtimestamp(timestamp)


def get_parse_state(raw_state, last_price, last_time):
    list = []
    price = raw_state["price"]
        
    list.append(raw_state["amount"])

    def prepare_orders(orders, price, multi):
        amount = float(orders[0][1])
        for order in orders:
            list.append((float(order[0])/price) * multi)
            list.append(float(order[1])/amount)

    history_step = 5
    bids = raw_state["bids"][:history_step]
    asks = raw_state["asks"][:history_step]
    prepare_orders(bids, price, 1)
    prepare_orders(asks, price, -1)

    if last_price != 0:
        list.extend([price/last_price])
    else:
        list.extend([0])
        
    current_timestamp = int(raw_state['timestamp'])
    if last_time != 0:
        list.extend([current_timestamp / last_time])
    else:
        list.extend([0])
    #list = []
    #list.extend([raw_state['timestamp']])
    #print(current_timestamp)
    return list

def get_future_state(df, sec=120): # 2 min in future
    state_timestamp = int(dt.get_from_index(dt.index-1)['timestamp'])
    timestamp_limit = state_timestamp + sec
    #print("Current timestamp", state_timestamp, " ==== ", timestamp_limit)
    index = 0
    timestamp = 0
    while timestamp < timestamp_limit:
        index += 3
        state = dt.get_from_index(dt.index + index)
        timestamp = int(state['timestamp'])
    return state



def get_state(raw_state):
    global last_price
    global last_time
    global should_buy
    global should_sell
    
    list = get_parse_state(raw_state, last_price, last_time)

    furure_state = get_future_state(dt)
    future_price = furure_state["price"]
    
    current_price = raw_state["price"]
    current_timestamp = int(raw_state['timestamp'])
    
    best_bid = float(furure_state["bids"][0][0]) 
    is_value_incresed = best_bid >= (current_price + 0.2)

    if is_value_incresed:
        should_buy += 1
        #print (current_price, " ==== ", (current_price + 0.2), " ===== ", furure_state)
        y = onehot_encoded(0)
    else:
        #print (current_price, " ==== ", (current_price + 0.2), " ===== ", furure_state)
        should_sell += 1
        y = onehot_encoded(1)
           
    #print (y)
    #print (get_date(raw_state), " ==== ", get_date(furure_state))
    
    last_price = current_price
    last_time = current_timestamp
    return [list, y]

In [5]:
from tqdm import tqdm

dt.rewind()

def get_set(data_count):
  trainX = []
  trainY = []
  for i in tqdm(range(data_count)):
      state = dt.next()
      state = get_state(state)
      trainX.append(state[0])
      trainY.append(state[1])
  trainX = np.array(trainX)
  trainY = np.array(trainY)
  return trainX, trainY

data_count = dt.steps - 1000
data_count = 40000

val_percentage = 0.1

trainX, trainY = get_set(int(data_count*(1-val_percentage)))

valX, valY = get_set(int(data_count*val_percentage))

print(trainX.shape)
print(trainY.shape)


print(valX.shape)
print(valY.shape)


100%|██████████| 3000/3000 [00:09<00:00, 314.83it/s]

(27000, 23)
(27000, 2)
(3000, 23)
(3000, 2)


In [6]:
# define generator
def get_gen(set_x, set_y, n_input = 200):
    return TimeseriesGenerator(set_x, set_y, length=n_input, batch_size=128*2)

train_generator = get_gen(trainX, trainY)

val_generator = get_gen(valX, valY)

# number of samples
print('Samples: %d' % len(train_generator))

x, y = train_generator[0]

features = x.shape[2]

Samples: 105


In [10]:

HP_NUM_SEQS = hp.HParam('num_seqs', hp.Discrete([10, 50, 100, 150]))
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([50, 100, 150]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.25))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['sgd', 'adam']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER, HP_NUM_SEQS],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )
  
def train_test_model(hparams):
  
    n_input = hparams[HP_NUM_SEQS]

    train_generator = get_gen(trainX, trainY, n_input)

    val_generator = get_gen(valX, valY, n_input)

    inputX = Input(shape=(n_input,features))
    
    def add_deep_layers(input_layer):
        x = Dense(100)(input_layer)
        x = tf.keras.layers.LeakyReLU(alpha=0.3)(x)
        x = Dropout(hparams[HP_DROPOUT])(x)
        x_shorcut = x
        x = Dense(100)(x)
        x = BatchNormalization()(x)
        x = tf.keras.layers.LeakyReLU(alpha=0.3)(x)
        x = Dense(hparams[HP_NUM_UNITS])(x)
        x = tf.keras.layers.LeakyReLU(alpha=0.3)(x)
        x = Dropout(hparams[HP_DROPOUT])(x)
        x = Dense(100)(x)
        x = tf.keras.layers.LeakyReLU(alpha=0.3)(x)
        x = layers.add([x, x_shorcut])
        return x


    lstm = add_deep_layers(LSTM(features*20)(inputX))
    x = add_deep_layers(lstm)

    x = add_deep_layers(x)
    x = add_deep_layers(x)
    x = Dense(hparams[HP_NUM_UNITS])(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.3)(x)
    x = Dropout(hparams[HP_DROPOUT])(x)
    x = Dense(100)(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.3)(x)
    x = Dense(2, activation='softmax')(x)

    model = Model(inputs=[inputX], outputs=x)

    filepath = "drive/My Drive/model/stock.h5"

    model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='categorical_crossentropy',
      metrics=['accuracy'],
    )

    model.fit_generator(train_generator, 
                          steps_per_epoch=len(train_generator)-1, 
                          epochs=10,
                          verbose=1)
    _, accuracy = model.evaluate(val_generator)
    return accuracy

In [11]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [ ]:
session_num = 0

for num_seqs in HP_NUM_SEQS.domain.values:
    for num_units in HP_NUM_UNITS.domain.values:
        for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
            for optimizer in HP_OPTIMIZER.domain.values:
                hparams = {
                HP_NUM_SEQS: num_seqs,
                HP_NUM_UNITS: num_units,
                HP_DROPOUT: dropout_rate,
                HP_OPTIMIZER: optimizer,
                }
                run_name = "run-%d" % session_num
                print('--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                run_name = 'logs/hparam_tuning/' + run_name
                run(run_name, hparams)
                session_num += 1


--- Starting trial: run-0
{'num_seqs': 10, 'num_units': 50, 'dropout': 0.1, 'optimizer': 'adam'}
Epoch 1/10
105/105 [==============================] - 33s 312ms/step - loss: 0.5904 - accuracy: 0.7446
Epoch 2/10
105/105 [==============================] - 17s 159ms/step - loss: 0.5820 - accuracy: 0.7501
Epoch 3/10
105/105 [==============================] - 18s 169ms/step - loss: 0.5799 - accuracy: 0.7582
Epoch 4/10
105/105 [==============================] - 17s 162ms/step - loss: 0.5755 - accuracy: 0.7578
Epoch 5/10
105/105 [==============================] - 17s 163ms/step - loss: 0.5599 - accuracy: 0.7615
Epoch 6/10
105/105 [==============================] - 17s 167ms/step - loss: 0.5631 - accuracy: 0.7533
Epoch 7/10
105/105 [==============================] - 16s 155ms/step - loss: 0.5658 - accuracy: 0.7554
Epoch 8/10
105/105 [==============================] - 37s 355ms/step - loss: 0.5451 - accuracy: 0.7562
Epoch 9/10
105/105 [==============================] - 18s 170ms/step - loss: 0.

105/105 [==============================] - 69s 658ms/step - loss: 0.5641 - accuracy: 0.7534
Epoch 5/10
105/105 [==============================] - 69s 657ms/step - loss: 0.5587 - accuracy: 0.7571
Epoch 6/10
105/105 [==============================] - 69s 658ms/step - loss: 0.5596 - accuracy: 0.7528
Epoch 7/10
105/105 [==============================] - 69s 660ms/step - loss: 0.5432 - accuracy: 0.7635
Epoch 8/10
105/105 [==============================] - 69s 658ms/step - loss: 0.5367 - accuracy: 0.7670
Epoch 9/10
105/105 [==============================] - 69s 658ms/step - loss: 0.5676 - accuracy: 0.7339
Epoch 10/10
12/12 [==============================] - 5s 383ms/step - loss: 0.5768 - accuracy: 0.7637
--- Starting trial: run-14
{'num_seqs': 50, 'num_units': 50, 'dropout': 0.25, 'optimizer': 'adam'}
Epoch 1/10
105/105 [==============================] - 119s 1s/step - loss: 0.5930 - accuracy: 0.7489
Epoch 2/10
105/105 [==============================] - 67s 643ms/step - loss: 0.5854 - accura

Epoch 1/10
105/105 [==============================] - 137s 1s/step - loss: 0.6123 - accuracy: 0.7392
Epoch 2/10
105/105 [==============================] - 69s 659ms/step - loss: 0.5900 - accuracy: 0.7545
Epoch 3/10
105/105 [==============================] - 70s 665ms/step - loss: 0.5748 - accuracy: 0.7598
Epoch 4/10
105/105 [==============================] - 70s 668ms/step - loss: 0.5764 - accuracy: 0.7488
Epoch 5/10
105/105 [==============================] - 70s 669ms/step - loss: 0.5607 - accuracy: 0.7622
Epoch 6/10
105/105 [==============================] - 70s 668ms/step - loss: 0.5657 - accuracy: 0.7526
Epoch 7/10
105/105 [==============================] - 70s 668ms/step - loss: 0.5655 - accuracy: 0.7584
Epoch 8/10
105/105 [==============================] - 71s 672ms/step - loss: 0.5548 - accuracy: 0.7574
Epoch 9/10
105/105 [==============================] - 70s 664ms/step - loss: 0.5735 - accuracy: 0.7447
Epoch 10/10
12/12 [==============================] - 5s 380ms/step - loss: 

In [10]:
%tensorboard --logdir logs/hparam_tuning

Reusing TensorBoard on port 6006 (pid 41488), started 6:45:08 ago. (Use '!kill 41488' to kill it.)

In [ ]:
#tf.keras.utils.plot_model(model, 'multi_input_and_output_model.png', show_shapes=True)

In [ ]:


checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='loss',
                             verbose=1,
                             save_best_only=True,
                             load_weights_on_restart=True)

callbacks_list = []

history = model.fit_generator(train_generator, 
                              validation_data=val_generator, 
                              validation_steps=len(val_generator)-1,
                              steps_per_epoch=len(train_generator)-1, 
                              epochs=20, verbose=1, 
                              callbacks=callbacks_list)


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
import datetime
print (datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))